In [1]:
import numpy as np
import pandas as pd
import time
import sys
import os
from joblib import Parallel, delayed
sys.path.append('../')
from packages import actv_analysis, svm, load_csv, stats, objects
import pickle

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
net=1; relu=5; epoch=0
# Load the list of objects from the file
with open(f'network{net}_Relu{relu}_epoch{epoch}.pkl', 'rb') as f:
    loaded_objects = pickle.load(f)

In [ ]:
numbers = np.arange(2,21,2)
sizes = np.arange(4,14)

def process_unit_parallel(net, relu, epoch, numbers, sizes):
    pickle_filename = f'network{net}_Relu{relu}_epoch{epoch}.pkl'
    actv_net = actv_analysis.get_actv_net(net,relu,epoch)
    avg_actv_net = np.mean(actv_net, axis=2)
    avg_actv_nxs = avg_actv_net.reshape(avg_actv_net.shape[0], len(numbers), len(sizes))
    units = None
    try:
        with open(pickle_filename, 'rb') as f:
            units = pickle.load(f)
            print(f'network{net}_Relu{relu}_epoch{epoch} is loaded')
        for unit in units:
            unit.PN = unit.find_PN(avg_actv_nxs)
            unit.PS = unit.find_PS(avg_actv_nxs)
        with open(pickle_filename, 'wb') as f:
            pickle.dump(units, f)
        print(f'network{net}_Relu{relu}_epoch{epoch} is clean')
    except Exception as e:
        print(f'An error occurred while processing file {pickle_filename}: {e}')
    finally:
        if units is not None:
            del units  # make sure we free up memory
            
_ = Parallel(n_jobs=-1, verbose=50)(delayed(process_unit_parallel)(net, relu, epoch, numbers, sizes) for net in range(1,11) for relu in range(4,6) for epoch in range(0,91,10))

# for net in range(1,2):
#     for relu in range(5,6):
#         for epoch in range(0,1,10):
#             process_unit_parallel(net,relu,epoch,numbers,sizes)

In [ ]:
# Define the parameters
for net in range(1, 11):  # replace with the actual range
    for relu in range(4, 6):  # replace with the actual range
        for epoch in range(0, 91, 10):  # replace with the actual range
            # Run the function in parallel for each combination of network, layer, and epoch
            stats.complete_anova2(net, relu, epoch)

In [ ]:
net=1; relu=4; epoch=10
min_sz_idx=3; max_sz_idx=9
numbers=np.arange(2,21,2)
inst=500

stats.complete_anova2(net, relu, epoch)

In [ ]:
net=1; relu=4; epoch=20
print(f'Network{net} Relu{relu} Epoch{epoch}')

pickle_filename = f'network{net}_Relu{relu}_epoch{epoch}.pkl'

if os.path.exists(pickle_filename):
    with open(pickle_filename, 'rb') as f:
        units = pickle.load(f)
    incomplete_units = np.array([i for i in range(len(units)) if not units[i].no_response_subset and (units[i].anova2 is None or units[i].anova2.isna().all().all())])
    
if len(incomplete_units) > 0:
    # setup other parameters as required by add_anova2 function
    min_sz_idx=3
    max_sz_idx=9
    numbers=np.arange(2,21,2)
    inst=500

    # prepare data
    actv_net = actv_analysis.get_actv_net(net=net, relu=relu, epoch=epoch)
    take = np.arange(0,100).reshape(10,10)[:,min_sz_idx:max_sz_idx+1].reshape(len(numbers)*(max_sz_idx-min_sz_idx+1))
    actv_szAtoB = actv_net[:,take,:]
    actv_2D = actv_szAtoB.reshape(actv_szAtoB.shape[0], actv_szAtoB.shape[1]*actv_szAtoB.shape[2])
    sizes = np.arange(4,14)[min_sz_idx: max_sz_idx+1]

    for unit_idx in incomplete_units:
        try:
            # run add_anova2 for the unit
            units[unit_idx].add_anova2(actv_2D, numbers, sizes, inst, parallel=0)
            print(f'Added anova2 for unit {unit_idx}')
        except Exception as e:
            print(f"An error occurred with unit {unit_idx}: {e}")

    # save the updated units back into the pickle file after processing all units
    with open(pickle_filename, 'wb') as f:
        pickle.dump(units, f)
    print(f'Added anova2 for {len(incomplete_units)} units and saved to {pickle_filename}')
else:
    print("No incomplete units found.")

In [ ]:
net=1; relu=4
num_incomplete = []
for epoch in range(0,91,10):
    print(f'Network{net} Relu{relu} Epoch{epoch}')
    pickle_filename = f'network{net}_Relu{relu}_epoch{epoch}.pkl'
    if os.path.exists(pickle_filename):
        with open(pickle_filename, 'rb') as f:
            units = pickle.load(f)
            incomplete_units = np.array([i for i in range(len(units)) if not units[i].no_response_subset and (units[i].anova2 is None or units[i].anova2.isna().all().all())])
            num_incomplete.append(len(incomplete_units))

In [ ]:
num_incomplete

In [ ]:
tasks = [(net, relu, epoch) 
         for net in range(1, 2)
         for relu in range(4, 5)
         for epoch in range(50, 71, 10)]

# you could use Parallel to call this function multiple times in parallel
results = Parallel(n_jobs=-1, verbose=10)(delayed(stats.complete_anova2)(*t) for t in tasks)

In [2]:
for net in range(1,11):
    for relu in range(3,4):
        for epoch in range(0,91,10):
            stats.pkl_anova2